In [3]:
import sys # Python system library needed to load custom functions
import math # module with access to mathematical functions
import os # for changing the directory

import numpy as np  # for performing calculations on numerical arrays
import pandas as pd  # home of the DataFrame construct, _the_ most important object for Data Science

from numpy import save  

from IPython.display import Audio # for listening to our insects
import IPython
from scipy.fft import fft # function to calculate Fast Fourier Transform

import torchaudio

import matplotlib.pyplot as plt  # allows creation of insightful plots
import seaborn as sns # another library to make even more beautiful plots

sys.path.append('../../src') # add the source directory to the PYTHONPATH. This allows to import local functions and modules.
# enable rendering plots under the code cell that created it
%matplotlib inline

from eda_utils import show_sampling, signal_generator, plot_random_spec, plot_spec, plot_waveform # functions to create plots for and from audio data
from gdsc_utils import download_directory, PROJECT_DIR # function to download GDSC data from S3 bucket and our root directory
from config import DEFAULT_BUCKET  # S3 bucket with the GDSC data

os.chdir(PROJECT_DIR) # changing our directory to root

In [3]:
download_directory('data/', None, DEFAULT_BUCKET)

In [4]:
cl= [1143,84,27,46,97,457,
               189,849,573,605,457,545,
               230,330,136,83,452,1060,
               40,1458,99,792,238,197,
               132,1108,620,536,573,1997,
               553,544,437,1288,779,96,
               603,215,329,1023,567,620,
               231,330,121,513,351,299,
               363,147,142,35,59,90,
               302,503,707,267,428,697,
               137,1138,576,96,286,109]

In [5]:
df = pd.read_csv('data/metadata.csv')

In [35]:
test_sort = test.query('label == 1')
#res1 = df.query('subset == "train" & length > 40 & label == 57') # just take train-data with length > 40
#res2 = df.query('subset == "validation" & label == 57')
#res3 = df.query('label == 57')

In [37]:
#groupby_sum1 = df_sort.groupby(['label']).sum() 
groupby_test = test_sort.groupby(['label']).sum()
#groupby_count1 = df_sort.count()
groupby_testcount = test_sort.count()
#groupby_count2 = res1.count()
#groupby_count3 = res2.count()
#groupby_count4 = res3.count()
print(groupby_testcount['label'])
anz_class = [1143, 84, 109]
print(anz_class)
#print(groupby_count1['length'])
#print(groupby_testcount['length'])
#print(groupby_count2['length'])
#print(groupby_count3['length'])
#print(groupby_count4['length'])

In [14]:
res = df.query('subset == "train" & length > 40') # just take train-data with length > 40

In [15]:
res_sorted = res.sort_values(by="label") # sort the train-data with length>40 by label/species

In [16]:
# VISUALIZATION:
# add variable that combines species and label
res_sorted['species and label'] = res_sorted.apply(lambda x: f"{x['species']} ({str(x['label'])})", axis = 1)
#res_sorted.head(2)

In [17]:
# Calculating stats per label/species - total length of recording per class and the total number of class occurences in the dataset
# sorted_stats shows how many seconds of recordings we have for each species, the number of different entries as well as the average length per sample (for trian-data)
sorted_stats = res_sorted.groupby(['label','species and label']).agg(length = ('length', 'sum'), count = ('species', 'count')).reset_index()

# Calculating average length of an audio sample
sorted_stats['avg_len'] = sorted_stats['length']/sorted_stats['count']
#sorted_stats.head()

In [8]:
sorted_stats = sorted_stats.sort_values('label')

plt.figure(figsize = (20,6))
sns.barplot(x = sorted_stats['species and label'], y = sorted_stats['count'], color = 'royalblue')
plt.title('Number of files per species (train-data) with length > 40 sec.', fontsize = 20)
plt.xticks(rotation = 90)
plt.show()

In [18]:
sorted_stats = sorted_stats.sort_values('label')

plt.figure(figsize = (20,6))
sns.barplot(x = sorted_stats['species and label'], y = sorted_stats['length'], color = 'royalblue')
plt.title('Length of files per species (train-data) with length > 40 sec.', fontsize = 20)
plt.xticks(rotation = 90)
plt.show()

In [146]:
# Exercise: Which species has the longest/shortest audio sample?
length_vec = df[["length", "species"]]
length_max = max(length_vec["length"])
length_min = min(length_vec["length"])
#print(length_max)
#print(length_min)
maxs = length_vec.query('length >= 707.0839909297052')
mins = length_vec.query('length <= 1.0')
print(maxs)
print(mins)

In [7]:
# Exercise: Which classes have most/least amount of data?
df['species and label'] = df.apply(lambda x: f"{x['species']} ({str(x['label'])})", axis = 1)
df_stats = df.groupby(['label','species and label']).agg(length = ('length', 'sum'), count = ('species', 'count')).reset_index()

amount_vec = df_stats[["species and label", "count"]]
amount_max = max(amount_vec["count"])
amount_min = min(amount_vec["count"])
#print(amount_max)
#print(amount_min)
#maxa = amount_vec.query('count >= 128')
#mina = amount_vec.query('count <= 8')
mina = amount_vec.query('count == 11')
#print(maxa)
print(mina)

In [3]:
ep_train_14_13 = 'data/train/Cicadaorni_GBIF3902038865_IN92382887_296104.wav'
plot_waveform(ep_train_14_13)

In [4]:
plot_spec([ep_train_14_13])

In [7]:
wave14_13, sample_rate = torchaudio.load(f'data/train/Cicadaorni_GBIF3902038865_IN92382887_296104.wav')
wave14_13_1 = wave14_13[0] # only one channel
wave14_13_1 = wave14_13_1[:2*sample_rate] 